In [6]:
import numpy as np
import pandas as pd

def load_sess_trials(cache, sess_id, stim_label_list, bin_size, kernel_param):
    # load session data
    data = cache.get_session_data(sess_id)
    for stim_i, stim_label in enumerate(stim_label_list):
        # get start, end time points
        stim_data_table = data.get_stimulus_table()[data.get_stimulus_table().stimulus_name==stim_label]
        start_time = np.array(stim_data_table.start_time)
        stop_time = np.array(stim_data_table.stop_time)
        # get windowed spike times
        S[i] = window_spk_times(data,start_time,stop_time)
        # get binned spike rates
        t_bin[i], P[i] = bin_spikes(S[i],bin_size)
        # kernel smoothed estimate of binned spike rates
        X[i] = smooth_spikes(P[i],kernel_param)
        
    return (t_bin, S, P, X)

In [7]:
def window_spk_times( data, start_time, stop_time ):
    # Vincent
    return S

In [8]:
def bin_spikes( S, bin_size, trial_time ):
    # Ryan
    P, t_bin = np.histogram(S, bins = np.arange(trial_time, step = bin_size))    return (t_bin, P)

In [9]:
def smooth_spikes( P, kernel_param ):
    # Rahul
    return X